In [1]:
import cv2
import numpy as np
import os
import shutil
import time

# Open the camera (using device 0; change to 1 if needed)
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to access the camera!")
    raise SystemExit

# Configuration parameters
interval = 1          # Capture interval in seconds
maxCycles = 10        # Circular buffer of 10 cycles (0-9)
count = 0             # Overall cycle counter

# Red line configuration
line_start = (430, 0)         # Starting point of the line (x, y)
line_end = (300, 475)         # Ending point of the line (x, y)
line_color = (0, 0, 255)      # Red color (BGR)
line_thickness = 5

# Resize and tile extraction parameters
targetWidth, targetHeight = 320, 240  # Resized image dimensions
tileSize = 80                         # Each tile is 80x80
stride = 40                           # Sliding window stride
expectedTileCount = 35                # Expected number of tiles

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture frame.")
            time.sleep(interval)
            continue

        # Draw the red line if coordinates are within frame bounds
        h, w, _ = frame.shape
        if (0 <= line_start[0] < w and 0 <= line_start[1] < h and
            0 <= line_end[0] < w and 0 <= line_end[1] < h):
            cv2.line(frame, line_start, line_end, line_color, line_thickness)
        else:
            print("Error: Line coordinates out of bounds.")

        # Clear pixels below the red line (set them to black)
        is_vertical = (line_end[0] == line_start[0])
        if not is_vertical:
            m = (line_end[1] - line_start[1]) / (line_end[0] - line_start[0])
            b_line = line_start[1] - m * line_start[0]
        else:
            m = 0
            b_line = 0

        cleared = frame.copy()
        rows, cols, _ = cleared.shape
        # Create a grid of coordinates
        Y, X = np.indices((rows, cols))
        if not is_vertical:
            y_line = m * X + b_line
            mask = Y > y_line
        else:
            mask = X > line_start[0]
        cleared[mask] = [0, 0, 0]

        # Determine the cycle index (0 to 9)
        cycle_index = count % maxCycles

        # Save the cleared image (will be overwritten on a new cycle)
        rawFilename = f"photo_{cycle_index}.jpg"
        cv2.imwrite(rawFilename, cleared)
        print(f"Saved cleared image: {rawFilename}")

        # Resize the cleared image to 320x240 and save it
        resized = cv2.resize(cleared, (targetWidth, targetHeight))
        resizedFilename = f"resized_{cycle_index}.jpg"
        cv2.imwrite(resizedFilename, resized)
        print(f"Saved resized image: {resizedFilename}")

        # Create (or replace) the folder for this cycle's tiles
        folderName = f"output_images_cycle_{cycle_index}"
        if os.path.exists(folderName):
            shutil.rmtree(folderName)
        os.makedirs(folderName)

        # Extract tiles from the resized image using a sliding window (stride = 40)
        tileCount = 0
        for row in range(0, resized.shape[0] - tileSize + 1, stride):
            for col in range(0, resized.shape[1] - tileSize + 1, stride):
                tile = resized[row:row+tileSize, col:col+tileSize]
                tileFilename = os.path.join(folderName, f"tile_{tileCount}.jpg")
                cv2.imwrite(tileFilename, tile)
                tileCount += 1

        if tileCount != expectedTileCount:
            print(f"Warning: Cycle {cycle_index} generated {tileCount} tiles (expected {expectedTileCount}).")
        else:
            print(f"Cycle {cycle_index} generated {tileCount} tiles.")

        time.sleep(interval)
        count += 1

except KeyboardInterrupt:
    print("Process terminated by user.")

cap.release()


[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Saved cleared image: photo_0.jpg
Saved resized image: resized_0.jpg
Cycle 0 generated 35 tiles.
Saved cleared image: photo_1.jpg
Saved resized image: resized_1.jpg
Cycle 1 generated 35 tiles.
Saved cleared image: photo_2.jpg
Saved resized image: resized_2.jpg
Cycle 2 generated 35 tiles.
Saved cleared image: photo_3.jpg
Saved resized image: resized_3.jpg
Cycle 3 generated 35 tiles.
Saved cleared image: photo_4.jpg
Saved resized image: resized_4.jpg
Cycle 4 generated 35 tiles.
Process terminated by user.
